<a href="https://colab.research.google.com/github/desireechen/Deep-Learning/blob/master/Natural_language_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Alternative 1. Dset that is in csv format.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2020-06-22 05:28:55--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 2607:f8b0:400c:c12::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.03s   

2020-06-22 05:28:55 (143 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [ ]:
# Alternative 2. Dset that is in json format. Get the entire dset. Then, separate training and validation.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

In [ ]:
# Alternative 3. Dset is in Tensorflow Datasets. Dset is separated into training and validation. Similar to Compilation 3 Alternative 2A.
import tensorflow_datasets as tfds
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

In [ ]:
# Alternative 4.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

In [ ]:
# Alternative 5. Two options below.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt

In [ ]:
# This is the 100-dimension version of GloVe from Stanford. This was used in Alternative 4.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [ ]:
# Alternative 1.
import csv
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]
sentences = []
labels = []
with open('/tmp/bbc-text.csv', 'r') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  for row in reader:
    labels.append(row[0])
    sentence = row[1]
    for word in stopwords:
      token = ' ' + word + ' '
      sentence = sentence.replace(token, ' ') # Replacing stopwords.
    sentences.append(sentence)

In [ ]:
# Alternative 2.
import json
with open('/tmp/sarcasm.json', 'r') as f:
  datastore = json.load(f)
sentences = []
labels = []
urls = []
for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

In [ ]:
# Alternative 3 is full IMDB. Alternative 3A is GRU, LSTM, Conv1D. Alternative 3B is IMDB subwords8K dataset.
import numpy as np
train_data, test_data = imdb['train'], imdb['test']
train_sentences = []
train_labels = []
test_sentences = []
test_labels =[]
for s,l in train_data:
  train_sentences.append(str(s.numpy())) # Depending on Python version, may need to use s.tonumpy instead of s.numpy
  train_labels.append(l.numpy())
for s,l in test_data:
  test_sentences.append(str(s.numpy())) # Depending on Python version, may need to use s.tonumpy instead of s.numpy
  test_labels.append(l.numpy())

In [ ]:
# Alternative 4. 
import csv
num_sentences = 0
corpus =[]

with open("/tmp/training_cleaned.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
      # Your Code here. Create list items where the first item is the text, found in row[5], and the second is the label. Note that the label is a '0' or a '4' in the text. When it's the former, make
      # your label to be 0, otherwise 1. Keep a count of the number of sentences in num_sentences
        list_item=[]
        list_item.append(row[5])
        label = row[0]
        if label == '4':
          list_item.append(1)
        else:
          list_item.append(0)
        num_sentences = num_sentences + 1
        corpus.append(list_item)

In [ ]:
# Alternative 5.
data = open('/tmp/irish-lyrics-eof.txt').read() # Input the file name accordingly.
corpus = data.lower().split("\n")

In [ ]:
# Alternatives 1, 2, 3 and 3A. For Alternative 3b, only use embedding_dim.
train_portion = 0.8
vocab_size = 10000 # This can be 1000 or 10000.
oov_tok = '<OOV>'
padding_type = 'post' # This can be 'post' or 'pre'.
trunc_type = 'post' # This can be 'post' or 'pre'.
max_length = 120 # This can be 16, 100 or 120.
embedding_dim = 16 # embedding_dim is used for NLP. This can be 16, 32 or 100.

In [ ]:
# Alternative 4.
train_portion = 0.9
# vocab_size = 10000 # This can be 1000 or 10000.
oov_tok = '<OOV>'
padding_type = 'post' # This can be 'post' or 'pre'.
trunc_type = 'post' # This can be 'post' or 'pre'.
max_length = 16 # This can be 16, 100 or 120.
embedding_dim = 100 # embedding_dim is used for NLP. This can be 16, 32 or 100.

subset_corpus_size = 160000 # I can choose to take all 160000 sentences or take a subset. 

In [ ]:
# Alternative 5.
# train_portion = 0.8
# vocab_size = 10000 # This can be 1000 or 10000.
# oov_tok = '<OOV>'
padding_type = 'pre' # This can be 'post' or 'pre'.
trunc_type = 'post' # This can be 'post' or 'pre'.
# max_length = 120 # This can be 16, 100 or 120.
embedding_dim = 100 # embedding_dim is used for NLP. This can be 16, 32 or 100.

In [ ]:
# Alternative 4.
import random

sentences = []
labels = []
random.shuffle(corpus)

for x in range(subset_corpus_size):
  sentences.append(corpus[x][0]) #x-th row and 1st column of corpus that has the sentence
  labels.append(corpus[x][1]) #x-th row and 2nd column of corpus that has the label

In [ ]:
# Alternative 1 and 4
train_size = int(train_portion * len(labels))
train_sentences = sentences[0:train_size]
test_sentences = sentences[train_size:]
train_labels = labels[0:train_size]
test_labels = labels[train_size:]

In [ ]:
# Alternative 2.
train_size = 20000
train_sentences = sentences[0:train_size]
test_sentences = sentences[train_size:]
train_labels = labels[0:train_size]
test_labels = labels[train_size:]

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok) # May not have num_words argument.
                                                    # Tokenizer does not care about punctuation and lower or upper case.
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)

In [ ]:
# Alternative 4.
tokenizer = Tokenizer() # Just initiate Tokenizer. There are no arguments.                                 
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) # Unique to Alternative 4.
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)

In [ ]:
# Alternative 5.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1 # Plus 1 for out-of-vocabulary words.

input_sequences = []
for line in corpus:
  sequences = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(sequences)):
    n_gram_sequence = sequences[:i+1]
    input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(x) for x in input_sequences])
input_padded = pad_sequences(input_sequences, padding=padding_type, maxlen=max_sequence_length)

import numpy as np
input_padded = np.array(input_padded) # For model training purposes, I need numpy arrays.
predictors, labels = input_padded[:,:-1], input_padded[:,-1]
labels = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [ ]:
# Not for Alternative 1.
import numpy as np
train_labels = np.array(train_labels) # For model training purposes, I need numpy arrays.
test_labels = np.array(test_labels) # For model training purposes, I need numpy arrays.
train_padded = np.array(train_padded) # For model training purposes, I need numpy arrays.
test_padded = np.array(test_padded) # For model training purposes, I need numpy arrays.

In [ ]:
# Extra code.
print(len(sentences))
print(len(labels))
print(sentences[2]) # Print the 3rd sentence.
print(padded[2]) # Print the padded sequences of the 3rd sentence.
print(padded.shape) # 2nd item shows how many tokens (or words) that the longest sentence has.

2225
tigers wary farrell  gamble  leicester say will not rushed making bid andy farrell great britain rugby league captain decide switch codes.   anybody else involved process still way away going next stage   tigers boss john wells told bbc radio leicester.  moment  still lot unknowns andy farrell  not least medical situation.  whoever take going take big  big gamble.  farrell  persistent knee problems  operation knee five weeks ago expected another three months. leicester saracens believed head list rugby union clubs interested signing farrell decides move 15-man game.  move across union  wells believes better off playing backs  least initially.  m sure make step league union involved centre   said wells.  think england prefer progress position back row can make use rugby league skills within forwards.  jury whether can cross divide.  club  balance will struck cost gamble option bringing ready-made replacement.
[5000 6977 3854 ...    0    0    0]
(2225, 2441)


In [ ]:
# Alternative 1. Need to tokenize the labels which are in text. 
print(len(word_index))
print('\nWord Index = ', word_index)
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
label_word_index = label_tokenizer.word_index
print('\nLabel Word Index = ', label_word_index)
train_label_sequences = np.array(label_tokenizer.texts_to_sequences(train_labels))
test_label_sequences = np.array(label_tokenizer.texts_to_sequences(test_labels))
# print(train_label_sequences)
# print(len(train_label_sequences))


Label Word Index =  {'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}
[[4], [2], [1], [1], [5], [3], [3], [1], [1], [5], [5], [2], [2], [3], [1], [2], [3], [1], [2], [4], [4], [4], [1], [1], [4], [1], [5], [4], [3], [5], [3], [4], [5], [5], [2], [3], [4], [5], [3], [2], [3], [1], [2], [1], [4], [5], [3], [3], [3], [2], [1], [3], [2], [2], [1], [3], [2], [1], [1], [2], [2], [1], [2], [1], [2], [4], [2], [5], [4], [2], [3], [2], [3], [1], [2], [4], [2], [1], [1], [2], [2], [1], [3], [2], [5], [3], [3], [2], [5], [2], [1], [1], [3], [1], [3], [1], [2], [1], [2], [5], [5], [1], [2], [3], [3], [4], [1], [5], [1], [4], [2], [5], [1], [5], [1], [5], [5], [3], [1], [1], [5], [3], [2], [4], [2], [2], [4], [1], [3], [1], [4], [5], [1], [2], [2], [4], [5], [4], [1], [2], [2], [2], [4], [1], [4], [2], [1], [5], [1], [4], [1], [4], [3], [2], [4], [5], [1], [2], [3], [2], [5], [3], [3], [5], [3], [2], [5], [3], [3], [5], [3], [1], [2], [3], [3], [2], [5], [1], [2], [2], [1],

In [ ]:
# Alternative 1 where there are 6 categories.
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             # tf.keras.layers.Dropout(0.5), # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                             # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
                             ## CHOOSE either one of the next two lines.
                             # tf.keras.layers.GlobalAveragePooling1D(), # Averages across the vector to flatten it out.
                             # tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(24, activation='relu'), # This number can be 6 or 24.
                             tf.keras.layers.Dense(6, activation='sigmoid')])
# model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # optimizer=tf.optimizers.Adam()

In [ ]:
# Alternatives 2, 3 and 3A.
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             # tf.keras.layers.Dropout(0.5), # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                             # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
                             ### tf.keras.layers.Conv1D(128, 5, activation='relu'), # 5 is the size of the filter. Number can be 64 or 128.
                             # CHOOSE either one of the next five options.
                             ### tf.keras.layers.GlobalAveragePooling1D(), # Averages across the vector to flatten it out.
                             ### tf.keras.layers.MaxPooling1D(pool_size=4),
                             # tf.keras.layers.Flatten(),
                             # tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)), # This is for GRU.
                             # tf.keras.layers.LSTM(64), # This number can be 32, 64, 100 or 150.
                             # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), # This is for LSTM. # This number can be 32, 64, 100 or 150.
                             tf.keras.layers.Dense(6, activation='relu'), # This number can be 6, 24 or 64.
                             tf.keras.layers.Dense(1, activation='sigmoid')])
# model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # optimizer=tf.optimizers.Adam()

In [ ]:
# Alternative 3B. Single-layer and Multi-layer LSTM. Also can do Conv1D with GlobalAveragePooling1D.
model = tf.keras.Sequential([tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
                             # tf.keras.layers.Dropout(0.5), # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                             # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
                             ### tf.keras.layers.Conv1D(128, 5, activation='relu'), # 5 is the size of the filter. Number can be 64 or 128.
                             # CHOOSE either one of the next five options.
                             ### tf.keras.layers.GlobalAveragePooling1D(), # Averages across the vector to flatten it out.
                             ### tf.keras.layers.MaxPooling1D(pool_size=4),
                             # tf.keras.layers.Flatten(),
                             # tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)), # This is for GRU.
                             # tf.keras.layers.LSTM(64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)), # This number can be 32, 64, 100 or 150.
                             # Need return_sequences for multi-layer LSTM.
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), # Single-layer LSTM. # This number can be 32, 64, 100 or 150.
                             tf.keras.layers.Dense(64, activation='relu'), # This number can be 6, 24 or 64.
                             tf.keras.layers.Dense(1, activation='sigmoid')])
# model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # optimizer=tf.optimizers.Adam()

In [ ]:
# Alternative 4.
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length,
                                                       weights=[embeddings_matrix], trainable=False),
                             tf.keras.layers.Dropout(0.2), # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                             # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
                             tf.keras.layers.Conv1D(64, 5, activation='relu'), # 5 is the size of the filter.
                             # CHOOSE either one of the next five options.
                             ### tf.keras.layers.GlobalAveragePooling1D(), # Averages across the vector to flatten it out.
                             tf.keras.layers.MaxPooling1D(pool_size=4),
                             # tf.keras.layers.Flatten(),
                             # tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)), # This is for GRU.
                             # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), # This is for LSTM. # This number can be 32, 64, 100 or 150.
                             tf.keras.layers.LSTM(64), # This number can be 32, 64, 100 or 150.
                             # tf.keras.layers.Dense(6, activation='relu'), # This number can be 6, 24 or 64.
                             tf.keras.layers.Dense(1, activation='sigmoid')])
# model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # optimizer=tf.optimizers.Adam()

In [ ]:
# Alternative 5.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras import regularizers
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length= max_sequence_length-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2)) # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                        # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
model.add(LSTM(100))
model.add(Dense(vocab_size/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(vocab_size, activation='softmax'))
# model.summary()

# EITHER do not wish to set learning rate of the Adam optimiser.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # optimizer=tf.optimizers.Adam()
# OR wish to set learning rate of the Adam optimiser.
optimizer = Adam(lr=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) # optimizer=tf.optimizers.Adam()

# Can have an early stop
# earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

In [ ]:
# Alternative 1.
num_epochs = 10 # This number can be 10, 30, 50 or 100.
history = model.fit(train_padded, train_label_sequences, epochs=num_epochs, validation_data=(test_padded, test_label_sequences), verbose=2)
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Alternatives 2, 3, 3A and 4.
num_epochs = 50 # This number can be 10, 30, 50 or 100.
history = model.fit(train_padded, train_labels, epochs=num_epochs, validation_data=(test_padded, test_labels), verbose=2)
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Alternative 3B.
num_epochs = 10 # This number can be 10, 30, 50 or 100.
history = model.fit(train_data, epochs=num_epochs, validation_data=test_data, verbose=1)
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Alternative 5.
num_epochs = 100 # This number can be 10, 30, 50 or 100.
history = model.fit(predictors, labels, epochs=num_epochs, verbose=1)
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Extra code.
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_sentence(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_sentence(train_padded[2])) # This is the 3rd padded train sentence.
print(train_sentences[2]) # This is the original sentence including punctuation, capital letters and OOV.
print(train_labels[2])

e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # In the format of (vocab_size, embedding_dim) showing how many vocabulary words and dimensions.

In [ ]:
# Extra code.
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  # Write out the words.
  out_m.write(word + "\n")
  # Write out the coefficients of each dimension on the vector for the word.
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [ ]:
# Extra code. Test a given sentence.
# EITHER THIS. sentences = ['I really think this is amazing. honest.']
# OR THIS. sentences = ["granny starting to fear spiders in the garden might be real", "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentences)
print('\n Sequence(s) = ', sequences)
print('\nWord Index = ', word_index) # Print this for reference.
padded = pad_sequences(sequences, padding=padding_type, truncating=trunc_type, maxlen=max_length)
print('\nPadded:')
print(model.predict(padded))

In [ ]:
import matplotlib.pyplot as plt
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss") # Lower loss means less confidence in the prediction.

In [ ]:
# Extra code.
model.save("test_bidirectional_LSTM.h5")
model.save("test_1Dconvolutional_layer.h5")

In [ ]:
# Extra code. Predict from given seed text.
seed_text = "Help me Obi Wan Kenobi, you're my only hope" # Can change the seed text.
next_words = 100 # Probably can use less words. 
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre') # Pad the seed text, so that it matches the model.
	predicted = model.predict_classes(token_list, verbose=0) # Use the model to predict what comes after the seed text. 
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)